# Edit domain_cfg.nc and mesh_mask.nc to fill lakes and problematic ice shelves

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cmocean
import cartopy.crs as ccrs

In [2]:
home ='/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/bathymetry/'

domcfg_orig   = xr.open_dataset(f'{home}domain_cfg-nico-test.nc')
meshmask_orig = xr.open_dataset(f'{home}mesh_mask-nico-test.nc')

In [3]:
def fill_lakes(array, dim='2D'):

    if dim=='2D':
        array[0, 186, slice(128,135)] = 0
        array[0, slice(447,453), slice(848,850)] = 0
        array[0, slice(441,446), slice(848,853)] = 0
        array[0, slice(426,438), slice(856,870)] = 0
        array[0, slice(424,431), slice(863,872)] = 0
        array[0, 318, slice(1350,1352)] = 0
    elif dim=='3D':
        array[0, :, 186, slice(128,135)] = 0
        array[0, :, slice(447,453), slice(848,850)]   = 0
        array[0, :, slice(441,446), slice(848,853)]   = 0
        array[0, :, slice(426,438), slice(856,870)]   = 0
        array[0, :, slice(424,431), slice(863,872)]   = 0
        array[0, :, 318           , slice(1350,1352)] = 0
    else:
        raise Exception('Only set up for dimension 2D or 3D')

    return array

In [5]:
def fill_problem_iceshelf(array, dim='2D'):
    # make problem iceshelf land...
    if dim=='2D':
        array[0, 309, 1327] = 0
        array[0, 219, 706] = 0
        array[0, 220, 706] = 0
    elif dim=='3D':
        array[0, :, 309, 1327] = 0
        array[0, :, 219, 706] = 0
        array[0, :, 220, 706] = 0
    else:
        raise Exception('Only set up for dimension 2D or 3D')

    return array

In [ ]:
# ---- Domain config file ------------
domcfg_new   = domcfg_orig.copy()
for var in ['bottom_level', 'bathy_metry', 'isf_draft', 'top_level']:
    new_array       = fill_lakes(domcfg_new[var].values, dim='2D')
    new_array       = fill_problem_iceshelf(new_array, dim='2D')
    domcfg_new[var] = (('time_counter', 'y', 'x'), new_array)

domcfg_new.to_netcdf(f'{home}domain_cfg-nico-test-method.nc', unlimited_dims=['time_counter'])

# ---- Mesh mask file ------------
meshmask_new = meshmask_orig.copy()
for var in ['bathy_metry', 'mbathy', 'misf', 'mhw', 'hw', 'tmaskutil', 'umaskutil', 'vmaskutil']:
    new_array         = fill_lakes(meshmask_new[var].values, dim='2D')
    new_array         = fill_problem_iceshelf(new_array, dim='2D')
    meshmask_new[var] = (('time_counter', 'y', 'x'), new_array)

for var in ['tmask', 'umask', 'vmask', 'fmask']:
    new_array         = fill_lakes(meshmask_new[var].values, dim='3D')
    new_array         = fill_problem_iceshelf(new_array, dim='3D')
    meshmask_new[var] = (('time_counter', 'nav_lev', 'y', 'x'), new_array)

meshmask_new.to_netcdf(f'{home}mesh_mask-nico-test-method.nc', unlimited_dims=['time_counter'])